This is to demonstrate how to use SigOpt software for hyperparameter tuning:

* After importing necessary modules, first create a connection to SigOpt API, you will need an account that you can sign up here: <https://sigopt.com/edu> and get the client token under the tab **API Tokens**
* Do the normal steps of Bayesian Optimization up till the creation of the function model_tune() which takes in the hyperparameters and return the best validation AUC (sidenote: since everything will be recorded at SigOpt API and you can download it as csv file later, you can get rid of recording searched parameters on your own)
* Then **important**: create an experiment with the connection to SigOpt API specifying the experiment name, the hypreparameters searching space, the type of hyperparameters (int, float, categorical), and finally observation budget (which is the number of iterations). I have all types in here already, so you can just copy as needed
* You can print out the experiment id link so you can easily check out experiment results
* Then **important**: you will need an function to pass the parameters you created in the experiments to model_tune(). Assignments are the default results from experiments, so you need to call 'assignments' dictionary for different hyperparameters
* Then **important**: create iterations(observations), the API will automatically update suggestions based on best result
* Then fetch the best experiment results after all iterations are done, but you can also manually retrieve the result at the experiment id link above
* Additional infor that might be need: delete an experiment (free trial account has 10 experiments limit, so be sure to manage your unwanted ones
* Model parameter by SigOpt: RNN with GRU cell, SigOpt gives the best validation AUC of 0.807
* Hyperparameters tested: dropout: **0.5519603420905197**; embdim_exp: **9**; eps_exp: **-5**; hid_exp: **9**; l2_exp: **-5**; layers_n: **1**; lr_exp: **-2**; opt_code: **Adam**  [click here](https://sigopt.com/guest?guest_token=FQAAQHWMAMOEWSVMBOTLESAPXZOGXZQSJNOUQCSPZQNSXFCZ)
* Local result: using the same set of parameters, run the RNN model locally on server, it gives us a result of **0.794 best validation auc**, with corresponding **0.796 test auc**. Difference exists but minor.

In [1]:
from __future__ import print_function
from __future__ import division

import string
import re
import random

import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchviz import make_dot, make_dot_from_trace

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle


#from bayes_opt import BayesianOptimization
import pandas as pd

In [2]:
from sigopt import Connection
conn = Connection(client_token='QULCGRMSXQAMRSTUPJPLAHDCHJMEMLXPSPZSQMWNEYKHLYJF')

In [3]:
torch.cuda.set_device(0)

In [4]:
torch.backends.cudnn.enabled=False

In [5]:
random.seed(3)

In [6]:
import model_HPS_V2 as model #this changed
import Loaddata_final as Loaddata
import TrVaTe_V2 as TVT #This changed 

# check GPU availability
use_cuda = torch.cuda.is_available()
use_cuda

True

In [7]:
# Load data set and target values
set_x = pickle.load(open('Data/h143.visits', 'rb'), encoding='bytes')
set_y = pickle.load(open('Data/h143.labels', 'rb'),encoding='bytes')

"""
model_x = []
for patient in set_x:
    model_x.append([each for visit in patient for each in visit])  
    
"""
model_x = set_x  #this is for the rest of the models
    
merged_set= [[set_y[i],model_x[i]] for i in range(len(set_y))] #list of list or list of lists of lists
print("\nLoading and preparing data...")    
train1, valid1, test1 = Loaddata.load_data(merged_set)
print("\nSample data after split:")  
print(train1[1])
print("model is", 'RNN_GRU') #can change afterwards, currently on most basic RNN


Loading and preparing data...

Sample data after split:
[0, [[491, 129, 813, 149, 84, 85], [491, 129, 813, 61, 84, 85], [358, 61, 84, 85], [491, 1250, 813, 61, 84, 85], [499, 61, 84, 85], [499, 61, 84, 85], [8004, 61, 84, 85], [2914, 61, 84, 85], [2914, 61, 84, 85], [2499, 499, 61, 84, 85], [499, 61, 84, 85], [1250, 129, 813, 61, 84, 85], [1250, 61, 84, 85]]]
model is RNN_GRU


In [8]:
epochs = 100

In [9]:
#test on RNN with celltype plain RNN, with differnent optimizers also as an option 
def model_tune(embdim_exp, hid_exp, layers_n, dropout, l2_exp , lr_exp, eps_exp, opt_code):
    #little transformations to use the searched values
    embed_dim = 2** int(embdim_exp)
    hidden_size = 2** int(hid_exp)
    n_layers = int(layers_n)
    dropout = round(dropout, 4)
    l2 = 10** int(l2_exp)
    lr = 10** int(lr_exp)
    eps = 10** (eps_exp)

        
    """
   #dealing with categorical data
    if int(dlm_code)<3:
      if int(ct_code) ==1:
          cell_type='RNN'   
      elif int(ct_code) ==2:
          cell_type='LSTM'
      elif int(ct_code) ==3:
          cell_type='GRU'
      
    if int(dlm_code)==1:
        w_model='RNN'
        ehr_model = model.EHR_RNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type)
    elif int(dlm_code)==2:
        w_model='DRNN'
        ehr_model = model.DRNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type)

    if use_cuda:
        ehr_model = ehr_model.cuda(0)    
        
    """    
       
    ehr_model = model.EHR_RNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type = 'RNN')

    if use_cuda:
        ehr_model = ehr_model.cuda(0)   
    
    if opt_code == 'Adadelta':
        opt= 'Adadelta'
        optimizer = optim.Adadelta(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps) ## rho=0.9
    elif opt_code == 'Adagrad':
        opt= 'Adagrad'
        optimizer = optim.Adagrad(ehr_model.parameters(), lr=lr, weight_decay=l2) ##lr_decay no eps
    elif opt_code =='Adam':
        opt= 'Adam'
        optimizer = optim.Adam(ehr_model.parameters(), lr=lr, weight_decay=l2,eps=eps ) ## Beta defaults (0.9, 0.999), amsgrad (false)
    elif opt_code =='Adamax':
        opt= 'Adamax'
        optimizer = optim.Adamax(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps) ### Beta defaults (0.9, 0.999)
    elif opt_code =='RMSprop':
        opt= 'RMSprop'
        optimizer = optim.RMSprop(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps)                
    elif opt_code =='ASGD':
        opt= 'ASGD'
        optimizer = optim.ASGD(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### other parameters
    elif opt_code =='SGD':
        opt= 'SGD'
        optimizer = optim.SGD(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### other parameters
  
    """
    ehr_model = model.EHR_RNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type = 'RNN')

    if use_cuda:
        ehr_model = ehr_model.cuda(0)    
        
    optimizer = optim.Adamax(ehr_model.parameters(), lr=lr, weight_decay=l2,eps=eps)    
   
    """
     
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0
  
    for ep in range(epochs):
        current_loss, train_loss = TVT.train(train1, model= ehr_model, optimizer = optimizer, batch_size = 200)
        avg_loss = np.mean(train_loss)
        valid_auc, y_real, y_hat  = TVT.calculate_auc(model = ehr_model, data = valid1, which_model = 'RNN', batch_size = 200)
        if valid_auc > bestValidAuc: 
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            best_model= ehr_model
            #bestTestAuc, y_real, y_hat = TVT.calculate_auc(model = ehr_model, data = test1, which_model = w_model, batch_size = 200)

        if ep - bestValidEpoch > 12:
            break
      
  
     
    #fname= 'RNN'+'E'+str(embed_dim)+'RNN'+'L'+str(n_layers)+'H'+str(hidden_size)+'D'+str(dropout)+'Adamax'+'L'+str(lr)+'P'+str(l2)  
    #bmodel_pth='models/'#+fname
    bestTestAuc, y_real, y_hat = TVT.calculate_auc(model = best_model, data = test1, which_model = 'RNN', batch_size = 200)
    #torch.save(best_model, bmodel_pth)
    #buf = '|%f |%f |%d ' % (bestValidAuc, bestTestAuc, bestValidEpoch )
    
    #pFile= 'RNN'+'|'+str(embdim_exp)+'|'+'RNN'+'|'+str(layers_n)+'|'+str(hid_exp)+'|'+str(dropout)+'|'+'Adamax'+'|'+str(lr_exp)+'|'+str(l2_exp)+'|'+str(eps_exp)+ buf  
    #print2file(pFile, logFile)
    print('n/best validation auc: ' + str(bestValidAuc))
    print('n/best test auc: ' + str(bestTestAuc))
    return bestValidAuc

In [12]:
# create 3 different experiments for 3 different cells, keep the optimizer categorical 
experiment = conn.experiments().create(
  name='RNNGRU_AUC_Optimization',
  parameters=[
    dict(name='embdim_exp', type='int', bounds=dict(min=5, max=9)),
    dict(name='hid_exp', type='int', bounds=dict(min=5, max=9)),
    dict(name='layers_n', type='int', bounds=dict(min=1, max=3)),
    dict(name='dropout', type='double', bounds=dict(min=0.1000, max= 0.9000)),
    dict(name='lr_exp', type='int', bounds=dict(min=-7, max=-2)), 
    dict(name='l2_exp', type='int', bounds=dict(min=-7, max=-1)),
    dict(name='eps_exp', type='int', bounds=dict(min=-9, max=-4)),  
    dict(name='opt_code', type='categorical', categorical_values=[dict(name='Adadelta'), dict(name='Adagrad'),
                                                                  dict(name='Adam'), dict(name='Adamax'),dict(name='RMSprop'),
                                                                  dict(name='ASGD'), dict(name='SGD')]),
  ],
  observation_budget=300 #10~20 times number of parameters 
  )

In [13]:
print("Created experiment: https://sigopt.com/experiment/" + experiment.id);

Created experiment: https://sigopt.com/experiment/44631


In [10]:
# Evaluate your model with the suggested parameter assignments
def evaluate_model(assignments):
  return model_tune(assignments['embdim_exp'], assignments['hid_exp'],assignments['layers_n'],assignments['dropout'],
                    assignments['lr_exp'],assignments['l2_exp'],assignments['eps_exp'],assignments['opt_code'])

In [ ]:
#run the experiments, no need to modify
for i in range(300):
  suggestion = conn.experiments(experiment.id).suggestions().create()
  value = evaluate_model(suggestion.assignments)
  conn.experiments(experiment.id).observations().create(
    suggestion=suggestion.id,
    value=value,
  )

In [11]:
#Wrapping up the experiments and get the results
best_assignments_list = (
    conn.experiments('44631')
        .best_assignments()
        .fetch()
)
if best_assignments_list.data:
    best_assignments = best_assignments_list.data[0].assignments

In [12]:
best_assignments

Assignments({
  "dropout": 0.5519603420905197,
  "embdim_exp": 9,
  "eps_exp": -5,
  "hid_exp": 9,
  "l2_exp": -5,
  "layers_n": 1,
  "lr_exp": -2,
  "opt_code": "Adam"
})

In [13]:
#tuned model auc results:
evaluate_model(best_assignments)

n/best validation auc: 0.7941332843739122
n/best test auc: 0.7964534342513292


0.7941332843739122

In [ ]:
#delete unwanted experiments 
experiment = conn.experiments('someid').delete()